In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

In [2]:
%load_ext nb_mypy

Version 1.0.5


# The N-Queens-Problem as a CSP

In [3]:
CSP = tuple[list[str], set[int], set[str]] 

The function `create_csp(n)` takes a natural number `n` as argument and returns
a *constraint satisfaction problem* that encodes the 
[`n`-queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle).
    
A constraint satisfaction problem $\mathcal{P}$ is a triple of the form
$$ \mathcal{P} = \langle \mathtt{Vars}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
where 
- `Vars` is a set of strings which serve as *variables*.

  The idea is that $V_i$ specifies the column of the queen that is placed in row $i$.
    
- `Values` is a set of *values* that can be assigned 
  to the variables in $\mathtt{Vars}$.
  
  In the 8-queens-problem we will have $\texttt{Values} = \{1,\cdots,8\}$.
- `Constraints` is a set of formulas from first order logic.  
  Each of these formulas is  called a *constraint* of $\mathcal{P}$.
  There are two different types of constraints.
  * We have constraints that express that no two queens that are positioned in different rows share the same
    column.  To capture these constraints, we define
    $$\texttt{DifferentCol} := \bigl\{ \texttt{V}_i \not= \texttt{V}_j \bigm| i \in \{1,\cdots,8\} \wedge j \in \{1,\cdots,8\} \wedge j < i \bigr\}.$$
    Here the condition $j < i$ ensures that, for example,  while we have the constraint
    $\texttt{V}_2 \not= \texttt{V}_1$ we do not also have the constraint  $\texttt{V}_1 \not= \texttt{V}_2$, as the latter 
    constraint would be redundant if the former constraint had already been established.
  * We have constraints that express that no two queens positioned in different rows share the same 
    diagonal.  The queens in row $i$ and row $j$ share the same diagonal iff the equation
    $$ |i - j| = |V_i - V_j| $$
    holds.  The expression $|i-j|$ is the absolute value of the difference of the rows of the queens in row
    $i$ and row $j$,  while the expression $|V_i - V_j|$ is the absolute value of the difference of the
    columns of these queens.  To capture these constraints, we define
    $$ \texttt{DifferentDiag} := \bigl\{ |i  - j| \not= |\texttt{V}_i - \texttt{V}_j| \bigm| i \in \{1,\cdots,8\} \wedge j \in \{1,\cdots,8\} \wedge j < i \bigr\}. $$

In [4]:
def create_csp(n: int) -> CSP:
    S              = range(1, n+1)       
    # We use a list to store the variables so that the 
    # animation of backtracking is easier to follow
    Variables      = [ f'V{i}' for i in S ]
    Values         = set(S)
    DifferentCols  = { f'V{i} != V{j}' for i in S
                                       for j in S
                                       if  i < j 
                     }
    DifferentDiags = { f'abs(V{j} - V{i}) != {j - i}' for i in S
                                                      for j in S 
                                                      if  i < j 
                     }
    return Variables, Values, DifferentCols | DifferentDiags

The function `main()` creates a CSP representing the 4-queens puzzle and prints the CSP.
It is included for testing purposes.

In [5]:
def main() -> None:
    Vars, Values, Constraints = create_csp(4)
    print('Variables:  ', Vars)
    print('Values:     ', Values)
    print('Constraints:')
    for c in Constraints:
        print('            ', c)

In [7]:
main()

Variables:   ['V1', 'V2', 'V3', 'V4']
Values:      {1, 2, 3, 4}
Constraints:
             abs(V2 - V1) != 1
             abs(V3 - V1) != 2
             V1 != V4
             abs(V4 - V3) != 1
             V2 != V3
             abs(V4 - V1) != 3
             V2 != V4
             V1 != V3
             abs(V3 - V2) != 1
             abs(V4 - V2) != 2
             V1 != V2
             V3 != V4


## Displaying the Solution

The following code requires that you install `chess-problem-visuals`, which is a package provided by *Philipp Polland*.  This package can be installed into the environment `ai` with the following command:
```
pip install git+https://github.com/reclinarka/chess-problem-visuals
```
The following line performs this installation inside the jupyter notebook.
Once you have executed this line once, you can comment it.

In [8]:
!pip install git+https://github.com/reclinarka/chess-problem-visuals

  Cloning https://github.com/reclinarka/chess-problem-visuals to /private/var/folders/q9/qftgdjx91wx4s5jcqkfz5bd00000gn/T/pip-req-build-au799kxc
  Running command git clone --filter=blob:none --quiet https://github.com/reclinarka/chess-problem-visuals /private/var/folders/q9/qftgdjx91wx4s5jcqkfz5bd00000gn/T/pip-req-build-au799kxc
  Resolved https://github.com/reclinarka/chess-problem-visuals to commit 764a29b376fe9dd3cbb2623ce8740f73c6711fa4
  Preparing metadata (setup.py) ... done


Next, we have to import the package `chess_problem_visuals` that was installed in the line above.

In [9]:
import chess_problem_visuals as cpv

The function `show_solution(Solution, width)` takes a dictionary that contains a variable assignment that represents a solution to the 8-queens puzzle.  It displays this Solution on a chess board.
* `Solution` is a dictionary mapping the variables $\texttt{V}_i$ to numbers.
  If $\texttt{Solution[V}_i\texttt{]} = k$, then the queen in row $i$ is placed in column $k$.
* `width` specifies the size of the board as a percentage of the width of notebook.  

In [10]:
def show_solution(Solution: dict[str, int], n: int, width: str="50%") -> cpv.Board:
    Queens = [None for col in range(n)]
    board  = cpv.Board(n, html_width=width)
    for row in range(n):
        col = Solution.get('V'+str(row+1), None)
        if col:
            board.add_piece((col-1, row), 'Q')
    return board

In [11]:
Solution = {'V5': 1, 'V2': 2, 'V4': 6, 'V6': 3, 'V1': 7, 'V7': 5, 'V3': 4}

In [12]:
show_solution(Solution, 8)

In [14]:
Solution2 = {'V5': 1, 'V2': 2, 'V4': 6, 'V6': 3, 'V1': 7, 'V7': 5, 'V3': 4, 'V9':8, 'V8': 9}

In [15]:
show_solution(Solution2, 9, "40%")